# Ансамблевые модели

## Задача классификации 

В этом практическом задании вы научитесь работать с ансамблевыми моделями. Мы начнем с задачи классификации итальянского вина на предмет его пренадлежности к одному из трех видов. Загрузите датасет `Wine Data Database` с помощью функции `load_wine` из модуля `sklearn.datasets`.

In [1]:
from sklearn.datasets import load_wine

X, y = load_wine(return_X_y=True)

Модель случайного леса для классификации представлена классом `RandomForestClassifier` из модуля `sklearn.ensemble`. Конструктор этого класса содержит аргумент `n_estimators`, который соответствует колличеству базовых алгоритмов в случайном лесе. Целью этого задания будет настройка этого параметра. Сравните модели случайных лесов с различным числом базовых алгоритмов `{1, 5, 10, 20}`. Что происходит с качеством случайного леса на тестовых данных при увеличении этого числа? Ответом на это задание `answer1` является лучшая оценка качества модели, округленная до трех знаков после запятой. Используйте `accuracy` как метрику качества и скользящий контроль `cross_val_score` как метод оценки качества модели. Установите параметр `cv = StratifiedKFold(4)`. Возьмите среднее значение оценки качества. Для каждой из моделей случайного леса используете `random_state=42` при создании нового экземпляра.

### *РЕШЕНИЕ*

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
estimators = [1, 5, 10, 20]
for est in estimators:
    rf = RandomForestClassifier(n_estimators=est, random_state=42)
    model_rf = rf.fit(x_train, y_train)
    
    predictions = model_rf.predict(x_test)
    accuracy = accuracy_score(y_test, predictions)
    
    cv = StratifiedKFold(n_splits=4)
    cv_score = cross_val_score(model_rf, X, y, scoring='accuracy', cv=cv)
    print('Accuracy: {:.3f};'.format(accuracy), 'Esimator: {};'.format(est), 'CV Score: {:.3f}'.format(cv_score.mean()))

Accuracy: 0.926; Esimator: 1; CV Score: 0.826
Accuracy: 0.963; Esimator: 5; CV Score: 0.938
Accuracy: 0.963; Esimator: 10; CV Score: 0.966
Accuracy: 0.981; Esimator: 20; CV Score: 0.978


Далее сравните модель градиентного бустинга `GradientBoostingClassifier` из `sklearn.ensemble` с логистической регрессией `LogisticRegression` из `sklearn.linear_model` на этой выборке. Используете параметр `random_state=42` при создании экземпляров классов. Какая из моделей работает лучше? Приведите лучшую оценку, округленную до трех знаков после запятой, в качестве ответа `answer2` на это задание. Какие выводы из этого можно сделать?

### *РЕШЕНИЕ*

In [3]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

for est in estimators:
    gb = GradientBoostingClassifier(n_estimators=est, random_state=42)
    lr = LogisticRegression(random_state=42)
    
    model_gb = gb.fit(x_train, y_train)
    model_lr = lr.fit(x_train, y_train)
    
    predictions_gb = model_gb.predict(x_test)
    predictions_lr = model_lr.predict(x_test)
    
    accuracy_gb = accuracy_score(y_test, predictions_gb)
    accuracy_lr = accuracy_score(y_test, predictions_lr)
    
    print('Accuracy GB: {:.3f}'.format(accuracy_gb), 'Accuracy LR: {:.3f}'.format(accuracy_lr))

Accuracy GB: 0.611 Accuracy LR: 0.981
Accuracy GB: 0.907 Accuracy LR: 0.981
Accuracy GB: 0.926 Accuracy LR: 0.981
Accuracy GB: 0.944 Accuracy LR: 0.981


## Задача регрессии

Загрузите уже известную вам выборку `Boston House Prices` и разделите ее случайным образом на тренировочную и тестовую выборку. Для этого используете функцию `train_test_split` с параметрами `random_state=54` и `test_size=0.33`. Мы будем сравнивать 4 модели: `RandomForestRegressor`, `GradientBoostingRegressor` из `sklearn.ensemble`, а так же Гребневую регрессию и ЛАССО (`Ridge`, `Lasso` из `sklearn.linear_model`). Обучите каждую модель на тренировочной выборке с параметром `random_state=42` в конструкторе. Какая из моделей показывает наименьшее значение среднеквадратической ошибки на тестовых данных? В качестве ответа `answer3` приведите это значение, округленное до двух цифр после запятой.

### *РЕШЕНИЕ*

In [4]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

boston = load_boston()
x_train, x_test, y_train, y_test = train_test_split(boston.data, boston.target, test_size=0.33, random_state=54)

rm = RandomForestRegressor(random_state=42)
gb = GradientBoostingRegressor(n_estimators=100, random_state=42)
ridge = Ridge(random_state=42)
lasso = Lasso(random_state=42)

model_rm = rm.fit(x_train, y_train)
model_gb = gb.fit(x_train, y_train)
model_ridge = ridge.fit(x_train, y_train)
model_lasso = lasso.fit(x_train, y_train)

predictions_rm = model_rm.predict(x_test)
predictions_gb = model_gb.predict(x_test)
predictions_ridge = model_ridge.predict(x_test)
predictions_lasso = model_lasso.predict(x_test)

mse_rm = mean_squared_error(y_test, predictions_rm)
mse_gb = mean_squared_error(y_test, predictions_gb)
mse_ridge = mean_squared_error(y_test, predictions_ridge)
mse_lasso = mean_squared_error(y_test, predictions_lasso)

print('MSE RM: {:.2f}'.format(mse_rm))
print('MSE GB: {:.2f}'.format(mse_gb))
print('MSE Ridge: {:.2f}'.format(mse_ridge))
print('MSE Lasso: {:.2f}'.format(mse_lasso))

MSE RM: 10.28
MSE GB: 8.51
MSE Ridge: 23.80
MSE Lasso: 26.92
